In [3]:
!pip install pandarallel==1.6.5
!pip install gensim


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 41.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 22.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 19.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.1
    Uninstalling scipy-1.15.1:
      Successfully uninstalled scipy-1.15.1

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import json
import regex as re
import tqdm.auto as tqdm
import pandas as pd
import string
import numpy as np
from glob import glob
import re
import string

from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
from nltk.tokenize import word_tokenize
from nltk import word_tokenize as lib_tokenizer

In [ ]:
wiki_20220301 = pd.read_parquet("/kaggle/input/wiki-20220301/wikipedia_20220301.parquet")
wiki_20220301.head(3)

In [ ]:
wiki_20220301 = wiki_20220301.fillna("NaN")

In [ ]:
dict_map = dict({})

def word_tokenize1(text):
    global dict_map
    words = text.split()
    words_norm = []
    for w in words:
        if dict_map.get(w, None) is None:
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace("``", '"').replace("''", '"')
        words_norm.append(dict_map[w])
    return words_norm


def strip_answer_string(text):
    text = text.strip()
    while text[-1] in '.,/><;:\'"[]{}+=-_)(*&^!~`':
        if text[0] != '(' and text[-1] == ')' and '(' in text:
            break
        if text[-1] == '"' and text[0] != '"' and text.count('"') > 1:
            break
        text = text[:-1].strip()
    while text[0] in '.,/><;:\'"[]{}+=-_)(*&^!~`':
        if text[0] == '"' and text[-1] != '"' and text.count('"') > 1:
            break
        text = text[1:].strip()
    text = text.strip()
    return text

def strip_context(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

def post_process(x):
    x = " ".join(word_tokenize1(strip_context(x))).strip()
    x = x.replace("\n", " ")
    x = "".join([i for i in x if i not in string.punctuation])
    return x

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=10)

In [ ]:
wiki_20220301_parts = np.array_split(wiki_20220301, 3)

In [ ]:
 wiki_20220301_parts[0]['text'] = wiki_20220301_parts[0]['text'].apply(lambda x: x.lower()).parallel_apply(post_process)


In [ ]:
wiki_20220301_parts[1]['text'] = wiki_20220301_parts[1]['text'].apply(lambda x: x.lower()).parallel_apply(post_process)

In [ ]:
wiki_20220301 = pd.concat(wiki_20220301_parts, ignore_index=True)
wiki_20220301.head(3)

In [ ]:
corpus = [x.split() for x in wiki_20220301['text'].values]

dictionary = Dictionary(corpus)
bm25_model = OkapiBM25Model(dictionary=dictionary)
bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]
bm25_index = SparseMatrixSimilarity(bm25_corpus, nums_docs=len(corpus), num_terms=len(dictionary), normalize_queries=False, normalize_documents=False)
tfidf_model = TfidfModel(dictionary=dictionary, smartins='bnn')

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import SparseMatrixSimilarity
from gensim.models.okapi_bm25 import OkapiBM25Model
import pandas as pd

# Đọc dữ liệu theo batch
def get_corpus_iter(df, batch_size=10000):
    for i in range(0, len(df), batch_size):
        yield [x.split() for x in df['text'].iloc[i:i+batch_size].values]

# Tạo từ điển từ corpus theo batch
dictionary = Dictionary()
for batch in get_corpus_iter(wiki_20220301):
    dictionary.add_documents(batch)

# Tạo BM25 Model với corpus stream
bm25_model = OkapiBM25Model(dictionary=dictionary)

# Ánh xạ corpus thành dạng bow theo batch
def bow_generator():
    for batch in get_corpus_iter(wiki_20220301):
        yield list(map(dictionary.doc2bow, batch))

bm25_corpus = bm25_model[bow_generator()]

# Tạo BM25 index
bm25_index = SparseMatrixSimilarity(
    bm25_corpus, num_docs=len(wiki_20220301), num_terms=len(dictionary),
    normalize_queries=False, normalize_documents=False
)

# Tạo TF-IDF Model trên streaming corpus
tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')


In [ ]:
dictionary.save("./kaggle/working/bm25/outputs/dict")
tfidf_model.save("./kaggle/working/bm25/outputs/tfidf")
bm25_index.save("./kaggle/working/bm25/outputs/bm25_index")